<a href="https://colab.research.google.com/github/thibaud-perrin/hibo-mistral-7b-fc/blob/main/test_hibo_mistral_7b_fc_vx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📖 hibo-mistral-7b-fc Testing

This notebook focuses on testing the newly saved model `thibaud-perrin/hibo-mistral-7b-fc-vx` from the Hugging Face Hub.

## 📦 Installation of Required Packages

Similar to the training notebook, we start by installing necessary packages that enable us to work with the model and perform evaluations. The `!pip install` command is used to ensure all dependencies are met for running the tests successfully.

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q trl xformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.8 MB/s eta 

## 📚 Import of All Required Packages

In this step, we import all the libraries and modules required for testing the model. This includes only `transformers`.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

## 🤖 Loading the New Model and Its Tokenizer

We load the `thibaud-perrin/hibo-mistral-7b-fc-v2` model along with its tokenizer from the Hugging Face Hub. This step is crucial for preparing the model for evaluation and ensuring it can process inputs correctly.

In [4]:
# Replace "AutoModelForCausalLM" and "AutoTokenizer" with specific model and tokenizer classes if necessary
model_identifier = "thibaud-perrin/hibo-mistral-7b-fc-v1.2"

model = AutoModelForCausalLM.from_pretrained(
    model_identifier,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)
tokenizer = AutoTokenizer.from_pretrained(model_identifier)

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.65k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [5]:
tokenizer.chat_template

"{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif false == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 and system_message != false %}{% set content = '<<SYS>>\n' + system_message + '\n<</

## 🔄 Put Model in Eval Mode

Before testing, we switch the model to evaluation mode using the `.eval()` method. This disables training-specific behaviors like dropout, ensuring the model's outputs are consistent and reflective of its true performance.

In [6]:
device = 'cuda:0'
# device = 'cpu'

In [7]:
model.config.use_cache = True
model.eval()
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

## 🧪 Test the Model

Finally, we test the model by running it on specific examples or a test dataset. This allows us to assess how well the model performs on the task it was fine-tuned for, such as instruction following or function calling. The outcomes of these tests can provide insights into any adjustments needed or confirm the model's readiness for deployment.

In [12]:
def stream(user_prompt):
    system_prompt = """You are a helpful assistant with access to the following functions. Use them if required -
    {
        "name": "get_stock_price",
        "description": "Get the current stock price of a company",
        "parameters": {
            "type": "object",
            "properties": {
                "company_name": {
                    "type": "string",
                    "description": "The name of the company"
                },
                "exchange": {
                    "type": "string",
                    "description": "The stock exchange where the company is listed"
                }
            },
            "required": [
                "company_name",
                "exchange"
            ]
        }
    }
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt.strip()}
    ]

    transformed_data = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    print(transformed_data)
    eos_token_id = tokenizer.eos_token_id
    tokenizer.pad_token = tokenizer.unk_token
    tokenizer.padding_side = "right"
    inputs = tokenizer([transformed_data], return_tensors="pt", add_special_tokens=False).to(device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    ids = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=512,
        eos_token_id=tokenizer.eos_token_id,
        early_stopping=True,
      )

In [16]:
stream("Hi, can you tell me the current stock price of Apple on NASDAQ? </s>")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST] <<SYS>>
You are a helpful assistant with access to the following functions. Use them if required -
    {
        "name": "get_stock_price",
        "description": "Get the current stock price of a company",
        "parameters": {
            "type": "object",
            "properties": {
                "company_name": {
                    "type": "string",
                    "description": "The name of the company"
                },
                "exchange": {
                    "type": "string",
                    "description": "The stock exchange where the company is listed"
                }
            },
            "required": [
                "company_name",
                "exchange"
            ]
        }
    }
    
<</SYS>>

Hi, can you tell me the current stock price of Apple on NASDAQ? </s> [/INST]
[ASST] <functioncall> {"name": "get_stock_price", "arguments": '{"company_name": "Apple", "exchange": "NASDAQ"}'} 
{"status": "success", "data": {"stock_pric